In [1]:
spark
import numpy as np
import pandas as pd
import re
import pyspark
from pyspark.sql.functions import lit
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import trim,ltrim,rtrim
from pyspark.sql.functions import *
from pyspark.sql.functions import mean
from ydata_profiling import ProfileReport
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import lit
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.sql.functions import mean, when, col
from pyspark.ml.classification import DecisionTreeClassificationModel
from pycaret.classification import *
from pycaret.classification import ClassificationExperiment
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from collections import Counter

Intitializing PySpark ...

23/05/30 14:29:18 WARN Utils: Your hostname, DESKTOP-VJTDOFP resolves to a loopback address: 127.0.1.1; using 172.27.220.32 instead (on interface eth0)
23/05/30 14:29:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/05/30 14:29:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Web UI available at http://172.27.220.32:4040
SparkContext available as 'sc' (version = 3.3.1, master = local[*], app id = local-1685453360741)
SparkSession available as 'spark'


In [2]:
path = "./model_data/part-00000-8f3d2c8d-e21b-4ae6-a337-4cbd28bb3ac8-c000.csv"

In [3]:

df = spark.read.option('header' , 'True').csv(path)

In [4]:
df.printSchema

<bound method DataFrame.printSchema of DataFrame[GameRound: string, nSets: string, Ground: string, Prize: string, GameRank_winner: string, GameRank_loser: string, Year: string, Height_dif: string, winner_home: string, loser_home: string, comb_hands: string, rank_dif: string]>

In [5]:
df.count()

76321

In [6]:
df = df.withColumn("nSets",df.nSets.cast('integer'))

In [7]:
df = df.withColumn("Prize",df.Prize.cast('integer'))

In [8]:
df = df.withColumn("Year",df.Year.cast('integer'))

In [9]:
df = df.withColumn("rank_dif",df.rank_dif.cast('double'))

In [10]:
df = df.withColumn("loser_home",df.loser_home.cast('int'))

In [11]:
df = df.withColumn("winner_home",df.winner_home.cast('int'))

In [12]:
df = df.withColumn("GameRank_winner",df.GameRank_winner.cast('double'))

In [13]:
df = df.withColumn("GameRank_loser",df.GameRank_loser.cast('double'))

In [14]:
df = df.withColumn("Height_dif",df.Height_dif.cast('double'))

In [15]:
df_pandas = df.toPandas()

report = ProfileReport(df_pandas) 
report.to_file("report_model.html")

In [16]:
setup = setup(df_pandas,target = "nSets",imputation_type= "iterative",fix_imbalance = True, normalize = True,session_id = 123,)

                        Description             Value
0                        Session id               123
1                            Target             nSets
2                       Target type            Binary
3                    Target mapping        2: 0, 3: 1
4               Original data shape       (76321, 12)
5            Transformed data shape       (94409, 31)
6       Transformed train set shape       (71512, 31)
7        Transformed test set shape       (22897, 31)
8                  Numeric features                 8
9              Categorical features                 3
10         Rows with missing values             33.9%
11                       Preprocess              True
12                  Imputation type         iterative
13  Iterative imputation iterations                 5
14        Numeric iterative imputer          lightgbm
15    Categorical iterative imputer          lightgbm
16         Maximum one-hot encoding                25
17                  Encoding

In [17]:
best = compare_models()

                                    Model  Accuracy     AUC  Recall   Prec.  \
gbc          Gradient Boosting Classifier    0.6695  0.5612  0.0067  0.5236   
dummy                    Dummy Classifier    0.6693  0.5000  0.0000  0.0000   
lightgbm  Light Gradient Boosting Machine    0.6684  0.5631  0.0205  0.4682   
ada                  Ada Boost Classifier    0.6675  0.5381  0.0126  0.2034   
catboost              CatBoost Classifier    0.6623  0.5583  0.0430  0.4018   
xgboost         Extreme Gradient Boosting    0.6566  0.5501  0.0703  0.3928   
rf               Random Forest Classifier    0.6311  0.5283  0.1386  0.3534   
knn                K Neighbors Classifier    0.5836  0.5173  0.2934  0.3468   
dt               Decision Tree Classifier    0.5596  0.5096  0.3617  0.3429   
lr                    Logistic Regression    0.4990  0.5574  0.6576  0.3593   
ridge                    Ridge Classifier    0.4986  0.0000  0.6584  0.3592   
lda          Linear Discriminant Analysis    0.4986 

In [18]:
best_models = compare_models(n_select=5, sort="AUC",exclude = ["dummy"])

                                    Model  Accuracy     AUC  Recall   Prec.  \
lightgbm  Light Gradient Boosting Machine    0.6684  0.5631  0.0205  0.4682   
gbc          Gradient Boosting Classifier    0.6695  0.5612  0.0067  0.5236   
catboost              CatBoost Classifier    0.6623  0.5583  0.0430  0.4018   
lr                    Logistic Regression    0.4990  0.5574  0.6576  0.3593   
lda          Linear Discriminant Analysis    0.4986  0.5573  0.6584  0.3592   
xgboost         Extreme Gradient Boosting    0.6566  0.5501  0.0703  0.3928   
ada                  Ada Boost Classifier    0.6675  0.5381  0.0126  0.2034   
nb                            Naive Bayes    0.3330  0.5335  0.9937  0.3308   
rf               Random Forest Classifier    0.6311  0.5283  0.1386  0.3534   
knn                K Neighbors Classifier    0.5836  0.5173  0.2934  0.3468   
qda       Quadratic Discriminant Analysis    0.3387  0.5157  0.9753  0.3306   
dt               Decision Tree Classifier    0.5596 

In [19]:
tuned_top5 = [tune_model(i, optimize= 'AUC') for i in best_models]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
      Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
Fold                                                          
0       0.6706  0.5651  0.0108  0.6129  0.0211  0.0098  0.0458
1       0.6704  0.5712  0.0130  0.5750  0.0255  0.0110  0.0451
2       0.6682  0.5555  0.0091  0.4211  0.0177  0.0039  0.0163
3       0.6700  0.5627  0.0136  0.5455  0.0265  0.0106  0.0416
4       0.6707  0.5660  0.0085  0.6818  0.0168  0.0087  0.0480
5       0.6709  0.5732  0.0079  0.7368  0.0157  0.0087  0.0516
6       0.6702  0.5661  0.0113  0.5714  0.0222  0.0095  0.0415
7       0.6687  0.5640  0.0102  0.4615  0.0199  0.0057  0.0238
8       0.6713  0.5689  0.0113  0.6667  0.0223  0.0114  0.0537
9       0.6711  0.5728  0.0091  0.6957  0.0179  0.0095  0.0510
Mean    0.6702  0.5665  0.0105  0.5968  0.0206  0.0089  0.0418
Std     0.0010  0.0051  0.0018  0.0973  0.0034  0.0022  0.0117
Fitting 10 folds for each of 10 candidates, totalling 1

In [20]:
blended_top5 = blend_models(estimator_list=tuned_top5)

      Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
Fold                                                          
0       0.6642  0.5591  0.0453  0.4278  0.0819  0.0198  0.0393
1       0.6622  0.5715  0.0430  0.4000  0.0777  0.0144  0.0283
2       0.6627  0.5539  0.0413  0.4033  0.0749  0.0144  0.0289
3       0.6702  0.5650  0.0447  0.5163  0.0823  0.0312  0.0677
4       0.6677  0.5678  0.0396  0.4730  0.0731  0.0232  0.0510
5       0.6655  0.5727  0.0407  0.4390  0.0746  0.0195  0.0409
6       0.6660  0.5685  0.0521  0.4577  0.0935  0.0278  0.0533
7       0.6630  0.5608  0.0424  0.4098  0.0769  0.0158  0.0316
8       0.6649  0.5735  0.0436  0.4326  0.0792  0.0199  0.0403
9       0.6653  0.5720  0.0442  0.4382  0.0802  0.0210  0.0425
Mean    0.6652  0.5665  0.0437  0.4398  0.0794  0.0207  0.0424
Std     0.0023  0.0063  0.0033  0.0336  0.0055  0.0052  0.0116


In [21]:
save_model(blended_top5, 'blended_top5') 

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('iterative_imputer',
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=IterativeImputer(add_indicator=False,
                                                                 cat_estimator=LGBMClassifier(boosting_type='gbdt',
                                                                                              class_weight=None,
                                                                                              colsampl...
                                                                  penalty='l2',
                                                                  random_state=123,
                                                                  solver='l

In [22]:
best_models = compare_models(n_select=5, 
                             sort="Accuracy",exclude = ["dummy"])

                                    Model  Accuracy     AUC  Recall   Prec.  \
gbc          Gradient Boosting Classifier    0.6695  0.5612  0.0067  0.5236   
lightgbm  Light Gradient Boosting Machine    0.6684  0.5631  0.0205  0.4682   
ada                  Ada Boost Classifier    0.6675  0.5381  0.0126  0.2034   
catboost              CatBoost Classifier    0.6623  0.5583  0.0430  0.4018   
xgboost         Extreme Gradient Boosting    0.6566  0.5501  0.0703  0.3928   
rf               Random Forest Classifier    0.6311  0.5283  0.1386  0.3534   
knn                K Neighbors Classifier    0.5836  0.5173  0.2934  0.3468   
dt               Decision Tree Classifier    0.5596  0.5096  0.3617  0.3429   
lr                    Logistic Regression    0.4990  0.5574  0.6576  0.3593   
ridge                    Ridge Classifier    0.4986  0.0000  0.6584  0.3592   
lda          Linear Discriminant Analysis    0.4986  0.5573  0.6584  0.3592   
svm                   SVM - Linear Kernel    0.4902 

In [23]:
tuned_top5 = [tune_model(i, optimize= 'Accuracy') for i in best_models]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).
      Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
Fold                                                          
0       0.6667  0.5354  0.0096  0.3542  0.0187  0.0013  0.0047
1       0.6678  0.5428  0.0074  0.3824  0.0144  0.0020  0.0088
2       0.6706  0.5400  0.0057  0.7692  0.0112  0.0064  0.0460
3       0.6691  0.5464  0.0153  0.4909  0.0296  0.0099  0.0347
4       0.6696  0.5538  0.0023  0.6667  0.0045  0.0023  0.0239
5       0.6692  0.5557  0.0011  0.5000  0.0023  0.0008  0.0098
6       0.6694  0.5563  0.0187  0.5077  0.0360  0.0129  0.0417
7       0.6655  0.5493  0.0119  0.3387  0.0230  0.0006  0.0018
8       0.6662  0.5537  0.0113  0.3509  0.0219  0.0013  0.0045
9       0.6689  0.5627  0.0113  0.4651  0.0221  0.0065  0.0258
Mean    0.6683  0.5496  0.0095  

In [24]:
blended_top5 = blend_models(estimator_list=tuned_top5)

      Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
Fold                                                          
0       0.6691  0.5681  0.0119  0.4884  0.0232  0.0076  0.0302
1       0.6702  0.5693  0.0192  0.5397  0.0372  0.0147  0.0485
2       0.6663  0.5549  0.0102  0.3462  0.0198  0.0009  0.0033
3       0.6702  0.5596  0.0153  0.5510  0.0297  0.0121  0.0451
4       0.6715  0.5639  0.0136  0.6667  0.0266  0.0136  0.0588
5       0.6703  0.5694  0.0130  0.5750  0.0255  0.0110  0.0451
6       0.6700  0.5685  0.0181  0.5333  0.0350  0.0136  0.0459
7       0.6692  0.5563  0.0147  0.5000  0.0286  0.0099  0.0357
8       0.6694  0.5694  0.0164  0.5000  0.0318  0.0110  0.0377
9       0.6709  0.5734  0.0136  0.6000  0.0266  0.0121  0.0497
Mean    0.6697  0.5653  0.0146  0.5300  0.0284  0.0106  0.0400
Std     0.0013  0.0060  0.0026  0.0797  0.0050  0.0038  0.0144


In [25]:
save_model(blended_top5, 'blended_top5_acc') 

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('iterative_imputer',
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=IterativeImputer(add_indicator=False,
                                                                 cat_estimator=LGBMClassifier(boosting_type='gbdt',
                                                                                              class_weight=None,
                                                                                              colsampl...
                                                             max_bin=None,
                                                             max_cat_threshold=None,
                                                             max_cat_to_onehot=

In [26]:
gbc = create_model("gbc")

      Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
Fold                                                          
0       0.6695  0.5541  0.0062  0.5238  0.0123  0.0046  0.0258
1       0.6700  0.5653  0.0057  0.6250  0.0112  0.0053  0.0343
2       0.6691  0.5490  0.0028  0.4545  0.0056  0.0015  0.0120
3       0.6704  0.5546  0.0091  0.6154  0.0178  0.0083  0.0423
4       0.6694  0.5621  0.0068  0.5217  0.0134  0.0049  0.0267
5       0.6692  0.5671  0.0040  0.5000  0.0079  0.0027  0.0184
6       0.6711  0.5674  0.0125  0.6471  0.0244  0.0121  0.0538
7       0.6683  0.5563  0.0091  0.4324  0.0177  0.0042  0.0180
8       0.6681  0.5679  0.0051  0.3600  0.0101  0.0008  0.0043
9       0.6698  0.5685  0.0057  0.5556  0.0112  0.0046  0.0278
Mean    0.6695  0.5612  0.0067  0.5236  0.0132  0.0049  0.0263
Std     0.0009  0.0068  0.0027  0.0868  0.0052  0.0031  0.0138


In [27]:
save_model(gbc, 'gbc_v1') 

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('iterative_imputer',
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=IterativeImputer(add_indicator=False,
                                                                 cat_estimator=LGBMClassifier(boosting_type='gbdt',
                                                                                              class_weight=None,
                                                                                              colsampl...
                                            criterion='friedman_mse', init=None,
                                            learning_rate=0.1, loss='log_loss',
                                            max_depth=3, max_features=None,
  

In [28]:
lr =  create_model ("lr")

      Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
Fold                                                          
0       0.4922  0.5466  0.6372  0.3521  0.4536  0.0480  0.0555
1       0.5104  0.5645  0.6570  0.3661  0.4702  0.0791  0.0910
2       0.4898  0.5430  0.6384  0.3509  0.4528  0.0454  0.0526
3       0.4956  0.5602  0.6701  0.3592  0.4677  0.0652  0.0769
4       0.4931  0.5583  0.6604  0.3563  0.4629  0.0582  0.0684
5       0.5079  0.5628  0.6576  0.3647  0.4692  0.0760  0.0877
6       0.5049  0.5620  0.6689  0.3646  0.4720  0.0765  0.0893
7       0.4989  0.5491  0.6486  0.3579  0.4613  0.0609  0.0705
8       0.5058  0.5707  0.6852  0.3673  0.4783  0.0840  0.0991
9       0.4910  0.5568  0.6529  0.3538  0.4589  0.0527  0.0617
Mean    0.4990  0.5574  0.6576  0.3593  0.4647  0.0646  0.0753
Std     0.0073  0.0082  0.0139  0.0058  0.0078  0.0130  0.0152


In [30]:
save_model(lr, 'lr_v1') 

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('iterative_imputer',
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=IterativeImputer(add_indicator=False,
                                                                 cat_estimator=LGBMClassifier(boosting_type='gbdt',
                                                                                              class_weight=None,
                                                                                              colsampl...
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=StandardScaler(copy=True,
                                                               with_mean=True,
                      

In [31]:
nb = create_model("nb")

      Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
Fold                                                          
0       0.3328  0.5274  0.9966  0.3310  0.4970  0.0009  0.0098
1       0.3309  0.5406  0.9955  0.3303  0.4960 -0.0013 -0.0168
2       0.3324  0.5235  0.9955  0.3308  0.4965  0.0002  0.0016
3       0.3322  0.5346  0.9960  0.3308  0.4966  0.0002  0.0017
4       0.3328  0.5378  0.9949  0.3309  0.4966  0.0003  0.0032
5       0.3326  0.5381  0.9949  0.3308  0.4965  0.0001  0.0014
6       0.3338  0.5312  0.9904  0.3307  0.4958 -0.0003 -0.0019
7       0.3373  0.5232  0.9819  0.3309  0.4950  0.0004  0.0022
8       0.3347  0.5420  0.9921  0.3311  0.4965  0.0014  0.0105
9       0.3306  0.5362  0.9994  0.3305  0.4968 -0.0002 -0.0070
Mean    0.3330  0.5335  0.9937  0.3308  0.4963  0.0002  0.0005
Std     0.0018  0.0065  0.0046  0.0002  0.0005  0.0007  0.0075


In [32]:
save_model(nb, 'nb_v1') 

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('iterative_imputer',
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=IterativeImputer(add_indicator=False,
                                                                 cat_estimator=LGBMClassifier(boosting_type='gbdt',
                                                                                              class_weight=None,
                                                                                              colsampl...
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=FixImbalancer(estimator=SMOTE(k_neighbors=5,
                                                                              n_jo

In [33]:
rf = create_model("rf")

      Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
Fold                                                          
0       0.6304  0.5352  0.1449  0.3556  0.2059  0.0178  0.0208
1       0.6285  0.5197  0.1285  0.3378  0.1861  0.0048  0.0057
2       0.6244  0.5253  0.1370  0.3343  0.1943  0.0025  0.0030
3       0.6337  0.5238  0.1341  0.3569  0.1950  0.0175  0.0210
4       0.6380  0.5378  0.1420  0.3752  0.2061  0.0298  0.0357
5       0.6247  0.5164  0.1341  0.3329  0.1912  0.0015  0.0017
6       0.6310  0.5201  0.1341  0.3496  0.1939  0.0128  0.0152
7       0.6380  0.5300  0.1409  0.3744  0.2048  0.0292  0.0350
8       0.6228  0.5357  0.1376  0.3306  0.1943  0.0000  0.0000
9       0.6398  0.5395  0.1529  0.3868  0.2192  0.0392  0.0463
Mean    0.6311  0.5283  0.1386  0.3534  0.1991  0.0155  0.0185
Std     0.0058  0.0080  0.0065  0.0190  0.0092  0.0130  0.0155


In [34]:
save_model(rf, 'rf_v1') 

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('iterative_imputer',
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=IterativeImputer(add_indicator=False,
                                                                 cat_estimator=LGBMClassifier(boosting_type='gbdt',
                                                                                              class_weight=None,
                                                                                              colsampl...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='sqrt',
              

In [35]:
lgbm = create_model("lightgbm")

      Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
Fold                                                          
0       0.6672  0.5640  0.0187  0.4286  0.0358  0.0084  0.0252
1       0.6697  0.5609  0.0226  0.5128  0.0434  0.0158  0.0471
2       0.6695  0.5581  0.0204  0.5070  0.0392  0.0140  0.0435
3       0.6698  0.5657  0.0226  0.5195  0.0434  0.0162  0.0485
4       0.6690  0.5596  0.0175  0.4921  0.0339  0.0114  0.0374
5       0.6690  0.5677  0.0249  0.4944  0.0474  0.0162  0.0453
6       0.6677  0.5681  0.0175  0.4429  0.0338  0.0088  0.0275
7       0.6717  0.5588  0.0221  0.6000  0.0426  0.0196  0.0635
8       0.6689  0.5665  0.0260  0.4842  0.0494  0.0162  0.0439
9       0.6690  0.5765  0.0159  0.4828  0.0307  0.0099  0.0339
Mean    0.6692  0.5646  0.0208  0.4964  0.0399  0.0136  0.0416
Std     0.0011  0.0053  0.0032  0.0441  0.0060  0.0036  0.0106


In [36]:
save_model(lgbm, 'lgbm_v1') 

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('iterative_imputer',
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=IterativeImputer(add_indicator=False,
                                                                 cat_estimator=LGBMClassifier(boosting_type='gbdt',
                                                                                              class_weight=None,
                                                                                              colsampl...
                 LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                colsample_bytree=1.0, importance_type='split',
                                learning_rate=0.1, max_depth=-1,
                      

In [37]:
lda = create_model("lda")

      Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
Fold                                                          
0       0.4977  0.5450  0.6333  0.3547  0.4547  0.0533  0.0612
1       0.5072  0.5651  0.6520  0.3634  0.4667  0.0730  0.0839
2       0.4832  0.5414  0.6333  0.3462  0.4477  0.0350  0.0408
3       0.5005  0.5597  0.6616  0.3608  0.4669  0.0681  0.0794
4       0.4931  0.5556  0.6582  0.3560  0.4621  0.0573  0.0672
5       0.5092  0.5636  0.6559  0.3653  0.4692  0.0771  0.0887
6       0.5019  0.5631  0.6802  0.3645  0.4746  0.0771  0.0910
7       0.4991  0.5499  0.6486  0.3580  0.4614  0.0612  0.0708
8       0.5047  0.5687  0.6772  0.3655  0.4748  0.0795  0.0934
9       0.5015  0.5585  0.6506  0.3596  0.4632  0.0651  0.0753
Mean    0.4998  0.5571  0.6551  0.3594  0.4641  0.0647  0.0752
Std     0.0071  0.0086  0.0148  0.0057  0.0080  0.0130  0.0152


In [38]:
save_model(lda, 'lda_v1') 

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('iterative_imputer',
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=IterativeImputer(add_indicator=False,
                                                                 cat_estimator=LGBMClassifier(boosting_type='gbdt',
                                                                                              class_weight=None,
                                                                                              colsampl...
                                                                              random_state=None,
                                                                              sampling_strategy='auto')))),
                ('normalize',
    

In [39]:
ct = create_model("catboost")

      Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
Fold                                                          
0       0.6594  0.5569  0.0419  0.3682  0.0752  0.0082  0.0157
1       0.6618  0.5549  0.0402  0.3901  0.0729  0.0118  0.0237
2       0.6639  0.5462  0.0464  0.4249  0.0837  0.0198  0.0387
3       0.6672  0.5549  0.0487  0.4699  0.0882  0.0279  0.0557
4       0.6629  0.5577  0.0424  0.4076  0.0769  0.0155  0.0308
5       0.6595  0.5627  0.0374  0.3587  0.0677  0.0056  0.0112
6       0.6565  0.5549  0.0379  0.3317  0.0681  0.0002  0.0004
7       0.6630  0.5520  0.0357  0.3962  0.0654  0.0114  0.0244
8       0.6614  0.5654  0.0515  0.4044  0.0914  0.0180  0.0329
9       0.6614  0.5696  0.0379  0.3785  0.0690  0.0093  0.0189
Mean    0.6617  0.5575  0.0420  0.3930  0.0758  0.0128  0.0253
Std     0.0028  0.0064  0.0050  0.0361  0.0087  0.0075  0.0147


In [40]:
save_model(ct, 'catboost_v1') 

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('iterative_imputer',
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=IterativeImputer(add_indicator=False,
                                                                 cat_estimator=LGBMClassifier(boosting_type='gbdt',
                                                                                              class_weight=None,
                                                                                              colsampl...
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=FixImbalancer(estimator=SMOTE(k_neighbors=5,
                                                                              n_jo

In [41]:
blender = blend_models([gbc,lgbm,lda,ct],optimize = "Accuracy")

      Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
Fold                                                          
0       0.6655  0.5631  0.0311  0.4231  0.0580  0.0133  0.0310
1       0.6650  0.5672  0.0272  0.4034  0.0509  0.0096  0.0233
2       0.6672  0.5554  0.0311  0.4545  0.0583  0.0166  0.0401
3       0.6708  0.5658  0.0340  0.5357  0.0639  0.0254  0.0638
4       0.6672  0.5658  0.0277  0.4495  0.0522  0.0144  0.0364
5       0.6672  0.5715  0.0266  0.4476  0.0502  0.0136  0.0352
6       0.6649  0.5686  0.0221  0.3861  0.0418  0.0062  0.0163
7       0.6668  0.5596  0.0215  0.4270  0.0409  0.0095  0.0266
8       0.6664  0.5724  0.0351  0.4429  0.0651  0.0173  0.0392
9       0.6672  0.5757  0.0266  0.4434  0.0502  0.0133  0.0341
Mean    0.6668  0.5665  0.0283  0.4413  0.0531  0.0139  0.0346
Std     0.0016  0.0058  0.0043  0.0377  0.0078  0.0050  0.0120


In [42]:
save_model(blender, 'blender_v1') 

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('iterative_imputer',
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=IterativeImputer(add_indicator=False,
                                                                 cat_estimator=LGBMClassifier(boosting_type='gbdt',
                                                                                              class_weight=None,
                                                                                              colsampl...
                                                              subsample_freq=0)),
                                              ('Linear Discriminant Analysis',
                                               LinearDiscriminantAnalysis(cova

In [43]:
blender_v2 = blend_models([gbc,lgbm,ct],optimize = "Accuracy")

      Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
Fold                                                          
0       0.6685  0.5640  0.0198  0.4730  0.0380  0.0117  0.0358
1       0.6708  0.5643  0.0198  0.5645  0.0383  0.0162  0.0538
2       0.6682  0.5552  0.0181  0.4571  0.0348  0.0099  0.0310
3       0.6693  0.5640  0.0153  0.5000  0.0297  0.0102  0.0364
4       0.6692  0.5635  0.0125  0.5000  0.0243  0.0084  0.0328
5       0.6703  0.5695  0.0187  0.5500  0.0361  0.0147  0.0497
6       0.6698  0.5657  0.0175  0.5254  0.0340  0.0129  0.0437
7       0.6703  0.5581  0.0164  0.5577  0.0319  0.0132  0.0478
8       0.6690  0.5693  0.0221  0.4875  0.0423  0.0140  0.0411
9       0.6689  0.5752  0.0125  0.4681  0.0243  0.0073  0.0275
Mean    0.6694  0.5649  0.0173  0.5083  0.0334  0.0118  0.0400
Std     0.0008  0.0054  0.0030  0.0370  0.0056  0.0027  0.0083


In [44]:
pred_v2 = predict_model(blender_v2)

               Model  Accuracy     AUC  Recall  Prec.  F1   Kappa     MCC
0  Voting Classifier    0.6704  0.5656       0      0   0  0.0143  0.0495


In [45]:
stacker = stack_models([gbc,lgbm,lda,ct])

      Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
Fold                                                          
0       0.6635  0.5667  0.0577  0.4340  0.1019  0.0263  0.0471
1       0.6594  0.5662  0.0656  0.4070  0.1131  0.0233  0.0385
2       0.6618  0.5560  0.0577  0.4180  0.1014  0.0230  0.0406
3       0.6631  0.5665  0.0656  0.4377  0.1142  0.0305  0.0520
4       0.6610  0.5640  0.0583  0.4120  0.1021  0.0219  0.0383
5       0.6587  0.5720  0.0549  0.3880  0.0962  0.0155  0.0270
6       0.6563  0.5667  0.0589  0.3755  0.1018  0.0133  0.0222
7       0.6621  0.5598  0.0538  0.4167  0.0952  0.0212  0.0385
8       0.6580  0.5715  0.0612  0.3899  0.1057  0.0177  0.0295
9       0.6642  0.5751  0.0674  0.4474  0.1171  0.0335  0.0568
Mean    0.6608  0.5664  0.0601  0.4126  0.1049  0.0226  0.0390
Std     0.0025  0.0054  0.0045  0.0221  0.0071  0.0060  0.0104


In [46]:
save_model(stacker,"stacker_v1")

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('iterative_imputer',
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=IterativeImputer(add_indicator=False,
                                                                 cat_estimator=LGBMClassifier(boosting_type='gbdt',
                                                                                              class_weight=None,
                                                                                              colsampl...
                                                 <catboost.core.CatBoostClassifier object at 0x7f1122c96c50>)],
                                    final_estimator=LogisticRegression(C=1.0,
                                                 